### ISyE524: Intro to Optimization
#### Problem Set #4
##### 1. Portfolio Optimization
###### 1-1

###### 1-2

###### 1-3

###### 1-4

##### 2. Making An Indefinite Matrix Definite
###### 2-1

In [4]:
using LinearAlgebra

# Define the matrix Q
Q = [
    0 0 -2 -4 0 1;
    0 1 -1 -1 3 -4;
    -2 -1 -1 -5 7 -4;
    -4 -1 -5 -3 7 -2;
    0 3 7 7 -1 -2;
    1 -4 -4 -2 -2 0
]

# Compute the eigenvalues and eigenvectors
eigenvalues = eigen(Q).values
# eigenvectors = eigen(Q).vectors
eigenvalues


6-element Vector{Float64}:
 -16.119094460644884
  -3.756648129364134
  -0.5922928569671964
   2.2331144580905455
   3.845741541835814
  10.389179447049864

###### 2-2

\begin{align*}
\text{Minimize} \quad & \sum_{i} D_{ii} \\
\text{Subject to:} \quad & x^{T}(Q+D)x≥0
\end{align*}

###### 2-3

In [6]:
using JuMP, SCS  
using LinearAlgebra

# Define the matrix Q
Q = [
    0 0 -2 -4 0 1;
    0 1 -1 -1 3 -4;
    -2 -1 -1 -5 7 -4;
    -4 -1 -5 -3 7 -2;
    0 3 7 7 -1 -2;
    1 -4 -4 -2 -2 0
]

# Find the smallest eigenvalue of Q
λ₁ = minimum(eigvals(Q))

# Create an optimization model
model = Model(SCS.Optimizer)  

# Add a variable for the diagonal matrix D
@variable(model, D[1:6] >= 0)

# The objective is to minimize the trace of D
@objective(model, Min, sum(D))

# Add the PSD constraint
@constraint(model, Q + Diagonal(D) in PSDCone())

# Solve the problem
optimize!(model)

# Check the solution status
status = termination_status(model)

if status == MOI.OPTIMAL
    # Fetch the optimal values of D
    optimal_D = value.(D)
    sum_optimal_D = sum(optimal_D)
    println("Optimal D: ", optimal_D)
    println("Sum of the diagonal elements of D: ", sum_optimal_D)
elseif status == MOI.INFEASIBLE
    println("The problem is infeasible.")
else
    println("The solver terminated with status: ", status)
end


------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 6, constraints m: 27
cones: 	  l: linear vars: 6
	  s: psd vars: 21, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 12, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.07e+01  3.37e+00  2.30e+02  9.47e+01  1.00e-01  9.92e-05 
   125| 6.91e-07  6.91e-07  5.93e-06  7.80e+01  1.00e-01  9.48e-04 
-------------------------------------------------

###### 2-4

\begin{aligned}
& \underset{X}{\text{minimize}}
& & \sum_{i=1}^{6}\sum_{j=1}^{6} |X_{ij}| \\
& \text{subject to}
& & Q + X \succeq 0, \\
&&& X = X^T.
\end{aligned}

###### 2-5

In [24]:
using JuMP, SCS, LinearAlgebra, PrettyTables

# Define the known Q matrix (you'll need to fill in the actual values of Q)
Q = [
    0 0 -2 -4 0 1;
    0 1 -1 -1 3 -4;
    -2 -1 -1 -5 7 -4;
    -4 -1 -5 -3 7 -2;
    0 3 7 7 -1 -2;
    1 -4 -4 -2 -2 0
]

# Create a model with the SCS solver
model = Model(SCS.Optimizer)

# Define the size of the matrix X
n = size(Q, 1)

# Define the variable for matrix X, ensuring it is symmetric
@variable(model, X[1:n, 1:n], Symmetric)

# Define auxiliary variables
@variable(model, Y[1:n, 1:n] >= 0)

# Add constraints to linearize the absolute value
for i in 1:n
    for j in 1:n
        @constraint(model, Y[i, j] >= X[i, j])
        @constraint(model, Y[i, j] >= -X[i, j])
    end
end

# objective function to use the auxiliary variables
@objective(model, Min, sum(Y[i, j] for i in 1:n for j in 1:n))

# Add the semidefinite constraint to the model
@constraint(model, Q + X in PSDCone())

# Solve the model
optimize!(model)

# Check the solution status
if termination_status(model) == MOI.OPTIMAL
    optimal_X = value.(X)
    println("Found optimal X:")
    pretty_table(optimal_X, header=string.(1:n), tf=tf_markdown)
else
    println("Could not find the optimal solution. Status: ", termination_status(model))
end

------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 57, constraints m: 129
cones: 	  l: linear vars: 108
	  s: psd vars: 21, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 201, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.25e+01  1.00e+00  3.14e+02 -1.30e+02  1.00e-01  2.56e-04 
   125| 8.96e-04  6.52e-05  3.84e-03  7.80e+01  1.00e-01  1.32e-03 
--------------------------------------------